In [2]:
!pip install lm_eval
# !pip install torch torchvision torchaudio llmcompressor
# !pip install transformers==4.53.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.3 MB/s eta 

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import lm_eval
from lm_eval import evaluator
from lm_eval.tasks import TaskManager
import os
import json
import numpy as np
import pickle
import shutil

2025-12-03 21:50:35.307679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764798635.521832      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764798635.587561      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## Замер метрик дефолтной модели с HuggingFace на бенчмарке MMLU

In [27]:
def evaluate_model(model_path="Qwen/Qwen3-8B"):
    
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    eval_model = lm_eval.models.huggingface.HFLM(
        pretrained=model,
        tokenizer=tokenizer,
        batch_size=4,
        max_length=2048,
        use_accelerate=True
    )
    
    total_params_compressed = sum(p.numel() for p in model.parameters())
    print(f"Количество параметров в сжатой модели: {total_params_compressed:,}")
    
    with torch.no_grad():
        results = evaluator.simple_evaluate(
            model=eval_model,
            tasks=["mmlu"],
            batch_size=4,
            num_fewshot=0,
            limit=None,
            device="cuda"
        )

    return results


def get_directory_size(path):
    """Calculates the total size of a directory in bytes."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

In [22]:
original_model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen3-8B",
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
original_model.save_pretrained("./qwen3-8b-original")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
results = evaluate_model()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

abstract_algebra/dev-00000-of-00001.parq(…):   0%|          | 0.00/3.45k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Running loglikelihood requests: 100%|██████████| 56168/56168 [29:33<00:00, 31.67it/s] 
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [14]:
# with open('default_metrics.pkl', 'wb') as file_handle:
#     pickle.dump(results, file_handle)
np.savez('default_metrics.npz', **results)

In [ ]:
data = np.load('default_metrics.npz', allow_pickle=True)

print("Доступные ключи:", list(data.keys()))
data_dict = {key: data[key] for key in data.keys()}
print("Загруженные данные:", data_dict)

Доступные ключи: ['results', 'groups', 'group_subtasks', 'configs', 'versions', 'n-shot', 'higher_is_better', 'n-samples', 'samples', 'config', 'git_hash', 'date', 'pretty_env_info', 'transformers_version', 'lm_eval_version', 'upper_git_hash', 'tokenizer_pad_token', 'tokenizer_eos_token', 'tokenizer_bos_token', 'eot_token_id', 'max_length']
Загруженные данные: 

In [18]:
results = data_dict['results'].item()
metrics = {}
for key, item in results.items():
    metrics[key] = item['acc,none']

In [28]:
print(f'Average accuracy of basic Qwen3-8B on whole MMLU benchmark: {metrics["mmlu"]}')
print(f'Basic Qwen3-8B size in bytes: {get_directory_size("/kaggle/working/qwen3-8b-original")}')
# shutil.rmtree('/kaggle/working/qwen3-8b-original')

Average accuracy of basic Qwen3-8B on whole MMLU benchmark: 0.7296681384418174
Basic Qwen3-8B size in bytes: 16381551095


## Компрессия модели

### 1. Начнем с квантизации

In [16]:
# !pip install torch torchvision torchaudio llmcompressor
# !pip install llmcompressor
# !pip install compressed-tensors
!pip install auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 62.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 106.2 MB/s eta 0:00:0000:010:01


In [18]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from datasets import load_dataset
# from llmcompressor.modifiers.quantization import GPTQModifier
# from llmcompressor.transformers import oneshot
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import load_dataset